# University scheduling problem
- University scheduling problem is a problem in which it is necessary to provide a student group with a professor and a classroom in a time intverval.

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#%pwd
#%cd gdrive/MyDrive/FAKS/OI/Projekat\ genetski\ algoritam

/content/gdrive/MyDrive/FAKS/OI/Projekat genetski algoritam


- School subject sessions - Genomes
- One instance of schedule - Chromosome
- Generated schedules - Population

In [ ]:
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import seaborn as sb
from threading import Thread
import concurrent.futures
rand.seed(42)

### Create all possible school_subject_sessions
- We are going to pick from this set

In [ ]:
def create_student_group_professor_permutation():
    school_subject_sessions = []
    for school_subject in School_subjects:
        list_to_permute = [school_subject.student_groups,school_subject.professors,school_subject.begin_end_times,Classrooms]
        list_permuted = list(itertools.product(*list_to_permute))
        for item_permuted in list_permuted:
            school_subject_sessions.append(School_subject_session(school_subject.id,item_permuted[0].id, school_subject.name,item_permuted[0],item_permuted[1],item_permuted[2],item_permuted[3]))
    return school_subject_sessions

In [ ]:
df = pd.DataFrame([vars(i) for i in create_student_group_professor_permutation()])

In [ ]:
df.to_csv('Schedule_permutations.csv')
print('amount of schedule subject session permutations', len(df))

amount of schedule subject session permutations 27450


In [ ]:
print('Number of school subject sessions to generate: {}'.format(len(df['id'].unique())))

Number of school subject sessions to generate: 45


# Genetic algorithm

##### Generate_chromosome - creates a single `schedule` with randomly picked schedule sessions
- 'chromosome' or a member of population

In [ ]:
def generate_chromosome(df):
    chromosome = []# a schedule
    for i in df['id'].unique():
        new_df = df[df['id'] == i]
        genome = new_df.loc[rand.randint(min(new_df.index),max(new_df.index))]# Subject_session instance
        chromosome.append( genome )
    chromosome = pd.DataFrame(chromosome)
    return chromosome

##### Initialize - creates a population (a number of schedules)

In [ ]:
def initialize(df, population_size):
    population = [] #schedules
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
      for i in range(0,population_size):
        futures.append(executor.submit(generate_chromosome, df))
      
      for i in range(0,population_size):
        population.append(futures[i].result())

    return population

#### Calculations for member fitness score
- Optimal fitnes score is low fitnes score, so we punish a member of a population for each mistake
- we simulate a graph colouring problem, if time overlaps by professor,student or classroom, it means that nodes are connected and share the same color

In [ ]:
def overlaps(genome,genomes_for_comparison):
    condition1 = (genome['start_time'] >= genomes_for_comparison['start_time']) & (genome['start_time'] < genomes_for_comparison['end_time'])
    condition2 = (genome['end_time'] > genomes_for_comparison['start_time']) & (genome['end_time'] <= genomes_for_comparison['end_time']) 
    df_overlapping = genomes_for_comparison.loc[(condition1 | condition2)]
    amount_overlapping = len(df_overlapping)
    return amount_overlapping*500

In [ ]:
def check_overlap(genomes):#genomes - currently selected subject sessions
    punishment = 0
    for genome_index in genomes.index:#part - subject session
        punishment += overlaps(genomes.loc[genome_index],genomes.drop(genome_index))
    return punishment

In [ ]:
def punshment_for_time_overlap_by_having_the_same(compare_item_column_name,member):
    overlap_punishment = 0
    for compare_item in member[compare_item_column_name].unique():
        df_compare_item = member[member[compare_item_column_name]==compare_item]
        overlap_punishment += check_overlap(df_compare_item)
        
    return overlap_punishment

In [ ]:
#not necessary for the task, but try to find a way to get genomes between certain hours
#def lower_time_punishment(member):
#    bad_time_genomes = member[(member.loc['start_time'].hour<10 | member.loc['start_time'].hour > 18 )]
#    return len(bad_time_genomes) * 10

In [ ]:
def fitness(member):
    punishment1 = punshment_for_time_overlap_by_having_the_same('professor',member)
    punishment2 = punshment_for_time_overlap_by_having_the_same('student_group',member)
    punishment3 = punshment_for_time_overlap_by_having_the_same('classroom',member)
    small_time_punishment = 1#lower_time_punishment(member) # let's say that everyone preferes classes in the morning
    punishment = punishment1 + punishment2 + punishment3 + small_time_punishment
    return punishment

#### Member breading

In [ ]:
def crossover(Member1,Member2):
    rand_float = rand.randint(20,50) / 100
    Member1_reset_index = Member1.reset_index()
    id_randomly_selected = Member1_reset_index.loc[int(len(Member1_reset_index)*rand_float)]['id']

    Member_1 = pd.concat([Member1[Member1['id'] < id_randomly_selected],Member2[Member2['id'] >= id_randomly_selected]])
    Member_2 = pd.concat([Member1[Member1['id'] > id_randomly_selected],Member2[Member2['id'] <= id_randomly_selected]])

    return Member_1,Member_2

#### Member mutations

In [ ]:
def mutate_by_id(member,id):
    df_with_this_id = df[df['id']==id]#svi sa ovim ID
    genome_with_this_id = member[member['id']==id]
    rand_num = rand.randint(1,len(df_with_this_id) -1)
    #40% probability this genome mutates
    if rand.randint(0,4) < 2:
      rand_number_sign = 1 if rand.randint(0,1) == 1 else -1
      rand_num = rand_num * rand_number_sign
      
      if (genome_with_this_id.index[0] + rand_num >= df_with_this_id.index.min()) and (genome_with_this_id.index[0] + rand_num <= df_with_this_id.index.max()):
          member = member.append(df_with_this_id.loc[genome_with_this_id.index[0] + rand_num])
          member = member.drop(genome_with_this_id.index[0])
          member.sort_values(by='id', inplace = True)

    return member

In [ ]:
def mutate(member,probability):

    rand_num = rand.randint(1,100)
    if rand_num < probability:
        for id in member['id'].unique():
            member = mutate_by_id(member,id)

    return member

#### Selection

In [ ]:
def pick_random_good_parent(generation_fitness_scores):
    rand_id = rand.randint(0,len(generation_fitness_scores) -1)
    random_parent_score = generation_fitness_scores[rand_id]
    num_of_times_to_choose = 3
    for i in range(num_of_times_to_choose):
        rand_id = rand.randint(0,len(generation_fitness_scores) -1)
        random_parent_score_2 = generation_fitness_scores[rand_id]
        if random_parent_score > random_parent_score_2:#lesser is better
            random_parent_score = random_parent_score_2
    return np.where(generation_fitness_scores == random_parent_score)[0]

In [ ]:
def pick_survivors(generation_fitness_scores) -> list:
    parents = set()
    
    while len(parents) < len(generation_fitness_scores)/2:
        parent_fitness_index = pick_random_good_parent(generation_fitness_scores)
        for parent_index in parent_fitness_index:
            parents.add(parent_index)

    return parents

In [ ]:
def get_generation_fitness_scores(population) -> list:
    current_generation_info = np.zeros(len(population))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [None] * len(population)
        for member_index in range(0,len(population)):
            futures[member_index] = executor.submit(fitness, population[member_index])
  
        for member_index in range(0,len(population)):
            score = futures[member_index].result()
            current_generation_info[member_index] = score
    return current_generation_info

# Genetic algorithm implementation

- Generating data frame by fitness for analysis

In [ ]:
def generate_df_by_fitness_of_population(fitness_of_the_population):
  data = pd.DataFrame(columns=['generation','member','fitness_score'])
  for generation in range(0,len(fitness_of_the_population)):
    zipped_gen = zip(*fitness_of_the_population[generation])
    gen_data_frame = pd.DataFrame(zipped_gen, columns=['generation','member','fitness_score'])
    data = pd.concat([data, gen_data_frame], ignore_index=True)
  return data

- genetic implementation

In [ ]:
def genetic(population_size,num_of_generations = 1000,num_of_elites = 4):
    fitness_of_the_population = []
    population = initialize(df,population_size)
    best_member = 0
    generation = 1

    generation_scores = get_generation_fitness_scores(population)
    fitness_of_the_population.append([[generation]*len(population), np.arange(0,len(population)) ,generation_scores])

    futures = [None] * len(population)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while True:

            print('Minimum fitness for generation {} equals: {}'.format(generation,generation_scores.min()))
            
            #Exit conditions
            if(generation_scores.min() == 1 or generation >= num_of_generations):
              best_member_index = np.argmin(generation_scores)
              best_member = population[best_member_index]
              break


            #saving the elites

            best_fitness_score_indexes = np.argpartition(generation_scores, num_of_elites)[:num_of_elites]
            elites = [population[index] for index in best_fitness_score_indexes]
            

            #selection
            survivor_indexes = pick_survivors(generation_scores)
            population = [population[i] for i in range(0,len(population)) if i in survivor_indexes]



            new_members = []
            for i in range(0,len(generation_scores)):
                rand1,rand2 = rand.randint(0,len(population)-1),rand.randint(0,len(population)-1)
                futures[i] = executor.submit(crossover, population[rand1],population[rand2])

            for i in range(0,len(generation_scores)):
                new_member1,new_member2 = futures[i].result()
                new_members.extend([new_member1,new_member2])

            
            #mutate elites (force mutations on elites)
            mutated_elites = elites.copy()
            for i in range(0,num_of_elites):
              futures[i] = executor.submit(mutate, mutated_elites[i],100)

            for i in range(0,num_of_elites):
              mutated_elites[i] = futures[i].result()
            
            population = elites
            population.extend(mutated_elites)
            population.extend(new_members[:population_size-num_of_elites*2])


            #Now mutate some of the other members
            for i in range(num_of_elites * 2,len(population)):
              futures[i] = executor.submit(mutate, population[i],20)

            for i in range(num_of_elites * 2,len(population)):
              population[i] = futures[i].result()


            generation += 1
            generation_scores = get_generation_fitness_scores(population)
            fitness_of_the_population.append([[generation]*len(population), np.arange(0,len(population)) ,generation_scores])



    return generate_df_by_fitness_of_population(fitness_of_the_population), best_member

#### Generating a with classes one after another to show that some classes need to overlap

In [ ]:
last_time = datetime.date.min

classes_overlapping = 0
indexes = []
for id in df['id'].unique():
    #print(df[df['id']==id])
    for index,row in df.loc[df['id']==id].iterrows():
        if row[4] >= last_time:
            last_time = row[5]
            indexes.append(index)
            break
        elif index == df.loc[df['id']==id].index[-1]:
            classes_overlapping +=1
            indexes.append(index)
            break
print('Classes overlapping: ',classes_overlapping)
print('Fitness score: ',fitness(df.loc[indexes]))
df.loc[indexes]

Classes overlapping:  27
Fitness score:  501001


,id,name,student_group,professor,start_time,end_time,classroom
0,13,Math 2,Data scientists,Igor,2022-01-01 08:00:00,2022-01-01 11:00:00,0
465,23,Introduction to physics,Data scientists,Igor,2022-01-01 11:00:00,2022-01-01 14:00:00,0
1155,25,Introduction to physics,Physicists 2,Igor,2022-01-01 14:00:00,2022-01-01 17:00:00,0
1845,26,Introduction to physics,Physicists 3,Igor,2022-01-02 08:00:00,2022-01-02 11:00:00,0
2540,34,Intermediate physics,Physicists 1,George,2022-01-02 11:00:00,2022-01-02 13:00:00,0
3050,35,Intermediate physics,Physicists 2,George,2022-01-02 13:00:00,2022-01-02 15:00:00,0
3560,37,Intermediate physics,Mechanical engeniers1,George,2022-01-02 15:00:00,2022-01-02 17:00:00,0
4065,44,Advanced physics,Physicists 1,George,2022-01-03 08:00:00,2022-01-03 11:00:00,0
4530,45,Advanced physics,Physicists 2,George,2022-01-03 11:00:00,2022-01-03 14:00:00,0
4995,46,Advanced physics,Physicists 3,George,2022-01-03 14:00:00,2022-01-03 17:00:00,0
